In [1]:
import cv2
import numpy as np
import pandas as pd

import pytesseract
from pytesseract import Output

import re

import skimage.filters as filters
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import PIL.Image

import os, random, pathlib, warnings, itertools, math, argparse, imutils
warnings.filterwarnings("ignore")

import import_ipynb
#from ipynb.fs.full.Extractor_string import img
#from ipynb.fs.full.data_exploring import orig1

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'E:\Programmes\Tesseract-OCR\tesseract.exe'

ap = argparse.ArgumentParser()
 
ap.add_argument("-i", "--image",
                required=True,
                help=r'path to image')
ap.add_argument("-c", "--min-conf",
                type=int, default=0,
                help="minimum confidence value to filter weak text detection")
args = vars(ap.parse_args())

In [ ]:
img = cv2.imread(r'C:\Users\LENOVO\Desktop\b.jpg')
#img = cv2.imread(args["image"])
plt.imshow(img)

In [4]:
def preprocessing_typing_detection(img):
    mask = np.zeros(img.shape, dtype=np.uint8)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    smooth = cv2.GaussianBlur(gray, (95,95), 0)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rgb1 = Image.frombytes('RGB', img.shape[:2], img, 'raw', 'BGR', 0, 0)
    division = cv2.divide(gray, smooth, scale=255)
    sharp = filters.unsharp_mask(division, radius=1.5, amount=1.5, channel_axis=False, preserve_range=False)
    sharp = (255*sharp).clip(0,255).astype(np.uint8)
    thresh = cv2.threshold(sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return thresh

def preprocessing_typing_detection1(inputImage):
    gray= cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)
    img= cv2.Laplacian(gray, cv2.CV_8U)
    return img

def preprocessing_typing_detection2(inputImage):
    image = imutils.resize(inputImage, height = 300)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(gray, 30, 200)
    return edged

def preprocessing_typing_detection3(inputImage):
    gray = cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    canny = cv2.Canny(blurred, 120, 255, 1)
    return canny

def preprocessing_typing_detection4(inputImage):
    gray = cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,128,255,cv2.THRESH_BINARY)
    thresh = cv2.bitwise_not(thresh)
    element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(7, 7))
    dilate = cv2.dilate(thresh,element,6)
    erode = cv2.erode(dilate,element,6)
    morph_img = thresh.copy()
    cv2.morphologyEx(src=erode, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img)
    return morph_img

def preprocessing_typing_detection5(inputImage):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Remove shadows
    dilated_img = cv2.dilate(gray, np.ones((7, 7), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = 255 - cv2.absdiff(gray, bg_img)
    norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255,
                            norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    # Threshold using Otsu's
    work_img = cv2.threshold(norm_img, 0, 255, cv2.THRESH_OTSU)[1]
    return work_img

def get_string(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((1,1), np.uint8)
    dil = cv2.dilate(gray, kernel, iterations = 1)
    img_f = cv2.erode(dil, kernel, iterations = 1)
    return img_f

In [5]:
thresh = preprocessing_typing_detection(img)
#img = preprocessing_typing_detection1(img)
#edged = preprocessing_typing_detection2(img)
#canny = preprocessing_typing_detection3(img)
morph_img = preprocessing_typing_detection4(img)
work_img = preprocessing_typing_detection5(img)
#img_f = get_string(img)

In [6]:
cv2.imshow('t',thresh)

In [ ]:
plt.imshow(thresh)

In [8]:
tessdata_dir_config = r'E:\Programmes\Tesseract-OCR\tesseract.exe\tessdata' 
tessdata_dir_config1 = r'E:\Programmes\Tesseract-OCR\tesseract.exe\tessdata_fast' 
tessdata_dir_config2 = r'E:\Programmes\Tesseract-OCR\tesseract.exe\tessdata_best' 
custom_config0 = r'--oem 3 --psm 6'
custom_config1 = r'--oem 3 --psm 11'
custom_config2 = r'--oem 3 --psm 8'
custom_config3 = r'--oem 1 --psm 3'

In [9]:
liste = [tessdata_dir_config, tessdata_dir_config1, tessdata_dir_config2, custom_config0, custom_config1, custom_config2, custom_config3]

In [10]:
# #[   0          1           2           3           4          5         6       7       8        9        10       11 ]
# #['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text']

In [11]:
def tessdata(img, par):
    #d = pytesseract.image_to_data(img)
    
    d = pytesseract.image_to_data(img, lang='eng', config=par, output_type=Output.DICT) #thresh
    
    return d

In [12]:
def tessdata1(img, par):
    text2 = pytesseract.image_to_string(img, lang='eng', config=par)  #mask or thresh 
    #print(text2) #thresh
    #return text2
    nums = re.findall(r'[A-Z][A-Z][0-9][0-9][0-9][0-9][0-9][0-9].|[A-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', text2)
    return nums

In [13]:
def spacebar(list):
    L = []
    for i in list:
        j = i.replace(' ','')
        L.append(j)
    return L

In [14]:
def extraction(img1, img2, par1, par2):
    d = tessdata(img1, par1)
    #print(d)
    nums = tessdata1(img2, par2)
    L = spacebar(nums)
    #print(L)
    #f = L[0]
    #text = None
    #conf = 0
    #M = 0,0,0,0
    return d, L

In [15]:
d, L = extraction(img,img,tessdata_dir_config1,tessdata_dir_config1)

In [ ]:
print(L)

In [17]:
def param(d, L):
    for i in range(0, len(d["text"])):
        x = d["left"][i]
        y = d["top"][i]
        w = d["width"][i]
        h = d["height"][i]

        text = d["text"][i]
        conf = int(float(d["conf"][i]))
        M = x, y, w, h
        
        #while conf > 40:
        #for i in L:
        if str(L[0]) == str(text) :
            return conf, text, M
            break
        else :
            continue

In [18]:
def box(conf, text, par):
    x, y, w, h = par
    text = "".join(text).strip()
    cv2.rectangle(img,
                (x, y),
                (x + w, y + h),
                (0, 0, 255), 2)
    cv2.putText(img,
                text,
                (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.2, (0, 255, 255), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    return("Confidence: {}".format(conf), "Text: {}".format(text))

In [19]:
def Localise():
    for a in liste:
        for b in liste : 
            d, L = extraction(img,img,a,b)
            print(L)
            if L !=[] :
                for i in range(0, len(d["text"])):
                    x = d["left"][i]
                    y = d["top"][i]
                    w = d["width"][i]
                    h = d["height"][i]
                    text = d["text"][i]
                    conf = int(float(d["conf"][i]))
                    M = x, y, w, h
                    if str(L[0]) == str(text) and (conf is not None) and (text is not None) and (M is not None) :
                        return (conf, text, M)                            
                        break
                    else :
                        continue
                    break
            else :
                continue

d, L = extraction(thresh,thresh,tessdata_dir_config,tessdata_dir_config)

In [ ]:
conf, text, M = Localise()

In [ ]:
print(M)

In [ ]:
box(conf, text, M)

In [ ]:
plt.imshow(img)